# Neural Model in Keras

In [ ]:
import os
import sys
import pickle as pkl

import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, Reshape, LSTM, Dense, Dropout
from keras.utils import multi_gpu_model, Sequence
from keras import optimizers
from keras.callbacks import TensorBoard

import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix

sys.path.append(os.path.abspath('../preprocessing/'))
from data_parser import DataParser
from config import Config

## Data preparation

### Load and balance data

In [ ]:
dp = DataParser(Config.embed_dim, Config.max_seq_len)
x_train_seq, x_train_syn, x_train_lex, y_train, embed_matrix = dp.parse_data()

In [ ]:
x_train_lex.shape

### Define Model

In [ ]:
seq_input = Input(shape=(Config.max_seq_len,), dtype='int32', name='seq_input')
syn_input = Input(shape=(Config.max_seq_len,), dtype='float32', name='syn_input')
lex_input = Input(shape=(4082,), dtype='float32', name='lex_input')

seq_embeds = Embedding(embed_matrix.shape[0], Config.embed_dim, 
                       weights=[embed_matrix], input_length=Config.max_seq_len,
                       trainable=False,
                       embeddings_regularizer=keras.regularizers.l2(.01),
                       name='embedding_layer')(seq_input)

syn_addons = Reshape((Config.max_seq_len, 1), name='reshape_layer')(syn_input)

time_series = seq_embeds
# time_series = Concatenate()([seq_embeds, syn_addons])

lstm_output = LSTM(Config.lstm_units, 
                   activation='relu',
                   name='lstm_layer')(time_series)

lex_feed = lex_input
for i in range(3):
    lex_feed = Dense(Config.lstm_units,
                 activation='relu',
                 kernel_regularizer=keras.regularizers.l2(.01),
                 name='lex_dense_layer_{}'.format(i))(lex_feed)
    lex_feed = Dropout(.01)(lex_feed)

concat = Concatenate()([lstm_output, lex_feed])

concat_feed = Dense(Config.lstm_units,
                    activation='relu',
                    kernel_regularizer=keras.regularizers.l2(.01),
                    name='concat_dense_layer_{}'.format(i))(concat)
concat_feed = Dropout(.01)(concat_feed)

predictions = Dense(Config.num_classes,
                    activation='softmax',
                    name='predictions')(concat_feed)

model = Model(inputs=[seq_input, syn_input, lex_input], outputs=predictions)

sgd = optimizers.SGD(lr=Config.learning_rate, decay=Config.learning_decay)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['acc'])
model.summary()

### Run model

In [ ]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

model.fit([x_train_seq, x_train_syn, x_train_lex], y_train, 
          batch_size=Config.batch_size, 
          epochs=Config.epochs, 
          validation_split=0.25,
          callbacks=[tensorboard])

y_pred_test = model.predict_on_batch([x_train_seq, x_train_syn, x_train_lex])
print confusion_matrix(y_train.argmax(axis=-1), y_pred_test.argmax(axis=-1))